In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

import hls4ml
import Test_layer as InstanceNormalization

from qkeras import *
from qkeras.quantizers import *

WARN: Unable to import optimizer(s) from expr_templates.py: No module named 'sympy'
Test layer registered in hls4ml


In [2]:
inputs = tf.keras.Input(shape=(20,20,1))
#x = QConv2D(filters=4, kernel_size=3, strides=1,
#                        groups=1,
#                         padding='same', 
#                         kernel_quantizer=quantized_bits(32,12,alpha=1),
#                         bias_quantizer=quantized_bits(32,12,alpha=1),
                         #kernel_regularizer=l2(0.001) , 
#                         name=f'dconv_b')(inputs)
x = tfa.layers.InstanceNormalization()(inputs)
model = tf.keras.Model(inputs=inputs, outputs=x)

2024-01-26 11:36:27.572568: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-26 11:36:27.577921: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-26 11:36:27.578250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-26 11:36:27.578754: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 20, 1)]       0         
                                                                 
 instance_normalization (Ins  (None, 20, 20, 1)        2         
 tanceNormalization)                                             
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [4]:
hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')

# Set the precision and reuse factor for the full model
hls_config['Model']['Precision'] = 'ap_fixed<16,6>'
hls_config['Model']['Strategy'] = 'Resource'
hls_config['Model']['ReuseFactor'] = 5

#hls_config['LayerName']['dconv_b']['Precision']['result'] = 'ap_fixed<32,12>'

Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 20, 20, 1]], output shape: [None, 20, 20, 1]
{'name': 'instance_normalization', 'class_name': 'Addons>InstanceNormalization', 'inputs': ['input_1'], 'data_format': 'channels_last', 'epsilon': 0.001}
Layer name: instance_normalization, layer type: Addons>InstanceNormalization, input shapes: [[None, 20, 20, 1]], output shape: [None, 20, 20, 1]


In [5]:
hls_config

{'Model': {'Precision': 'ap_fixed<16,6>',
  'ReuseFactor': 5,
  'Strategy': 'Resource',
  'BramFactor': 1000000000,
  'TraceOutput': False},
 'LayerName': {'input_1': {'Trace': False,
   'Precision': {'result': 'fixed<16,6>'}},
  'instance_normalization': {'Trace': False,
   'Precision': {'result': 'fixed<16,6>',
    'gamma': 'fixed<16,6>',
    'beta': 'fixed<16,6>'}}}}

In [6]:

cfg = hls4ml.converters.create_config(part='xcu250-figd2104-2L-e',backend='Vivado')
cfg['IOType']     = 'io_parallel' # Must set this if using CNNs!
cfg['HLSConfig']  = hls_config
cfg['KerasModel'] = model
cfg['OutputDir']  = 'Instnorm_test/'
cfg['XilinxPart'] = 'xcu250-figd2104-2L-e'
  
hls_model = hls4ml.converters.keras_to_hls(cfg)

Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 20, 20, 1]], output shape: [None, 20, 20, 1]
{'name': 'instance_normalization', 'class_name': 'Addons>InstanceNormalization', 'inputs': ['input_1'], 'data_format': 'channels_last', 'epsilon': 0.001}
Layer name: instance_normalization, layer type: Addons>InstanceNormalization, input shapes: [[None, 20, 20, 1]], output shape: [None, 20, 20, 1]
Creating HLS model


In [7]:
hls_model.compile()

Writing HLS project
Done


In [8]:
a = np.random.rand(1,20,20,1)

In [9]:
a.flatten().mean()

0.4775596433009066

In [10]:
model.get_weights()

[array([1.], dtype=float32), array([0.], dtype=float32)]

In [11]:
model.predict(a)

array([[[[-0.39372885],
         [ 1.0149423 ],
         [ 1.2211033 ],
         [-0.7926705 ],
         [-0.18489063],
         [ 0.9775764 ],
         [ 0.09591317],
         [-0.81143516],
         [-1.5594076 ],
         [-0.57916784],
         [-0.41390288],
         [-1.6538534 ],
         [-0.2351129 ],
         [-0.26069832],
         [-0.7896232 ],
         [-0.93080455],
         [-0.00728786],
         [ 1.3731409 ],
         [ 1.3579746 ],
         [-0.321445  ]],

        [[ 0.44248188],
         [ 0.31482923],
         [ 0.06634259],
         [ 1.5593761 ],
         [-0.61022806],
         [ 0.11493456],
         [ 1.6399039 ],
         [-0.6896024 ],
         [-0.4205793 ],
         [-0.34339917],
         [ 0.28451645],
         [-0.801831  ],
         [-0.98441   ],
         [-1.2393641 ],
         [ 1.3054589 ],
         [ 0.32797968],
         [ 0.51499736],
         [ 1.389912  ],
         [-1.6133881 ],
         [ 0.09435141]],

        [[ 1.3811237 ],
         [-0

In [12]:
a[0].shape

(20, 20, 1)

In [13]:
hls_model.predict(a)

array([-9.76562500e-04, -9.76562500e-04, -9.76562500e-04, -9.76562500e-04,
       -2.13593750e+01,  1.80546875e+01,  3.17763672e+01,  0.00000000e+00,
       -1.90000000e+01, -1.73046875e+01,  3.17773438e+01,  0.00000000e+00,
       -2.90468750e+01, -2.69833984e+01,  3.17773438e+01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        3.12500000e+01,  1.81074219e+01,  2.94384766e+01,  3.00097656e+01,
        1.05312500e+01, -2.09960938e+01,  3.17763672e+01,  0.00000000e+00,
        1.87500000e-01,  4.11132812e-01,  0.00000000e+00,  0.00000000e+00,
       -2.93281250e+01, -2.69833984e+01,  3.17773438e+01,  0.00000000e+00,
        3.87500000e+00,  1.10351562e-01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        2.26718750e+01, -2.05839844e+01,  3.17763672e+01,  0.00000000e+00,
        1.01875000e+01,  1.80556641e+01,  3.17763672e+01,  0.00000000e+00,
       -1.22871094e+01,  

In [14]:
hls_model.build()


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2020.1 (64-bit)
  **** SW Build 2902540 on Wed May 27 19:54:35 MDT 2020
  **** IP Build 2902112 on Wed May 27 22:43:36 MDT 2020
    ** Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.

source /home/marco/tools/Xilinx/Vivado/2020.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/home/marco/tools/Xilinx/Vivado/2020.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'marco' on host 'WattMachine2' (Linux_x86_64 version 5.15.0-92-generic) on Fri Jan 26 11:36:35 CET 2024
INFO: [HLS 200-10] On os Ubuntu 20.04.6 LTS
INFO: [HLS 200-10] In directory '/home/marco/NN-projects/hls4ml_forked/hls4ml/contrib/new_layer_template/Instnorm_test'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Opening project '/home/marco/NN-projects/hls4ml_forked/hls4ml/contrib/new_layer_template/Instnorm_test/myproject_prj'.
INFO: [HLS 200-10] Adding design f

{'CSimResults': [['0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0.015625',
   '0',
   '0',
   '0',
   '-6.66992',
   '25.1133',
   '0.00390625',
   '0',
   '-12.625',
   '0.0195313',
   '0',
   '0',
   '-8.625',
   '0.0195313',
   '0',
   '0',
   '-8.625',
   '0.0195313',
   '0',
   '0',
   '0.625',
   '0',
   '0',
   '0',
   '0.625',
   '0',
   '0',
   '0',
   '0.000976563',
   '0',
   '0',
   '0',
   '0.00390625',
   '0',
   '0.015625',
   '0',
   '0.00488281',
   '0',
   '19.5693',
   '0.0830078',
   '0.00195313',
   '-16',
   '0.00390625',
   '0',
   '0.00292969',
   '0',
   '0',
   '0',
   '0.00390625',
   '0',
   '0.0195313',
   '0',
   '0.00292969',
   '0',
   '19.5693',
   '0.0830078',
   '1.97363',
   '1.26172',
   '-31.5117',
   '-10.4248',
   '-22.1201',
   '23.3242',
   '-10.5176',
   '-19.4092',
   '22.084',
   '-23.4492',
   '0.00390625',
   '0',
   '0.015625',
   '0',
   '0.000976563',
   